# Deploying Prefect

### Introduction

Now that we've explored the main concepts in prefect, it's time to see if we can get this running on AWS.  To accomplish this, the we'll need to create an EC2 machine, and perform the installations necessary to boot up prefect.

### Creating the EC2 machine

Ok the first thing we should do is login to AWS, and create the EC2 machine.  You may need to save a new `.pem` file, which can be stored in your `~/.ssh` folder.  

Once you see your EC2 machine up and running, the next step is to connect to your machine, by clicking the connect button.

<img src="./connect.png">

Upon doing so, we'll see instructions telling us how to ssh into our machine, that look like the following.

<img src="./connect-to-instance.png">

> So you'll need to cd into the ssh folder, and make sure to run the `chmod 400` command on the pem file.  Then issue the `ssh` command that you see in your AWS environment, under example to login to the machine.

### Installing Prefect

Once we are connected to our machine, we can take the steps necessary to install prefect.

1. Install git and clone in our git repository

```bash
sudo yum update -y
sudo yum install git -y
git clone https://github.com/data-engineering-jigsaw/prefect-deployments-code.git
```

2. Get setup with pip, and install prefect

```bash
sudo yum install python3 -y
curl -O https://bootstrap.pypa.io/get-pip.py
python3 get-pip.py --user
```

At this point we can install prefect with the following:

```bash
pip3 install prefect
````

4. Connect prefect

And we'll want to connect prefect to our prefect cloud account.  To do so, go to prefect, and click on your profile in the bottom left, followed by API keys.

After creating an API key, you can copy and paste the `prefect cloud login` command into your AWS shell.

> <img src="./prefect-login.png" width="50%">

This will create a `~/.prefect` folder on your EC2 machine, and in the `~/.prefect/profiles.yml` file is your api key.

<img src="./prefect-profile.png">

Ok, the prefect runnning on our EC2 should now be connected to our prefect account.  Let's try it out.

Run the flow by moving into the `prefect-deployments-code` folder, and then running the `index.py` file.

<img src="./run-prefect.png" width="80%">

And if you click on the prefect link, we should see our flow run logged to the prefect orchestration environment in `prefect.io`.

### Deployments and Worker Pools

The flow run that we just created comes from the flow `get-restaurants`.  

> We can determine this because that's the name of the flow function.

If you click on the deployments panel, you may see that a `deployment` is already associated with this flow.

<img src="./existing-dep.png" width="70%">

If you look at the `get_restaurants-deployment.yaml` file associated with this, you'll see that we store the location folder of our flow code with the `path` key.

Prefect will autogenerate this for us.  But here, we have changed this to be a link to where our code is located in our AWS environment.

```yaml
storage: null
path: /home/ec2-user/prefect-deployments-code
```

This allows prefect cloud to download the flowcode from AWS machine.  Next, let's apply the deployment.

`prefect deployment apply get_restaurants-deployment.yaml`

> Confirm that there is a new deployment if you click on the `deployments` panel.

And we'll need to start the pool in the background.

```bash
prefect agent start -p 'default-agent-pool' &
```

So we add the `&` to have this be a background process, which will allow us to exit out of our EC2 machine and still have this run.

Then just type `exit` and press return to have the shell exit.

<img src="./dep-old-newest.png">

And we should see these deployments succeed even after we exit prefect.

Then we should stop our machine, and destroy it.

### Summary

In this lesson, we saw how to use an EC2 machine as our orchestration environment.  We did so by installing git, and the cloning our codebase, and installing prefect.

We then synced to our orchestrator by running `prefect cloud login` to add our prefect api keys to our EC2 machine.  We saw that prefect was then synced by running our flow.  And then, we reapplied our deployment, as we needed to update the path that contained our flow code.

Finally, we started our agent pool on our EC2 machine and set it to a background process.